In [ ]:
import pandas as pd

In [ ]:
# forest = pd.read_csv("models_hypotheses/Random_Forest_gpu_hypotheses_as_sentences.csv")
xgb = pd.read_csv("models_hypotheses/XGBoost_hypotheses_as_sentences.csv")
# rules = pd.read_csv("models_hypotheses/rules_df.csv")
lgb = pd.read_csv("models_hypotheses/LightGBM_hypotheses_as_sentences.csv")
lift = pd.read_csv("models_hypotheses/LIFT_hypotheses_as_sentences.csv")
lift.rename(columns={'Cancer Type': 'cancer_type'}, inplace=True)

In [ ]:
# Define source-to-method mapping
sources = {
    'lgb': 'LGBM',
    'xgb': 'XGBS',
    'lift': 'LIFT',
    # 'rules': 'RULE',
    # 'forest': 'FRST',
}

# Initialize an empty list to store processed dataframes
processed_dfs = []

# Process each dataframe in a loop
for df_name, df in [('xgb', xgb), ('lgb', lgb), ('lift', lift)]:
    # Create a copy to avoid modifying the original
    temp_df = df.copy()

    # Add method column
    temp_df['method'] = sources[df_name]

    # Rename the hypothesis column to 'hypo' as requested in output
    temp_df = temp_df.rename(columns={'hypothesis': 'hypo_factors'})

    # Add to list of processed dataframes
    processed_dfs.append(temp_df)

# Combine all processed dataframes
combined_df = pd.concat(processed_dfs, ignore_index=True)

combined_df.sort_values(by=['method', 'support'], ascending=[True, False], inplace=True)
combined_df['rank'] = combined_df.groupby('method')['support'].rank(ascending=False, method='first').astype(int)
combined_df['hypo_id'] = combined_df.apply(
        lambda row: f"{row['method']}.{row['cancer_type'].upper().replace(' ', '_')}.{row['rank']}",
        axis=1
    )

# Display the first few rows to verify the result
combined_df.head()

In [ ]:
# make the first column the hypo_id, the second the cancer_type, the third the hypo_factors while keeping all the other columns
combined_df = combined_df[['hypo_id', 'cancer_type', 'hypo_factors'] + [col for col in combined_df.columns if col not in ['hypo_id', 'cancer_type', 'hypo_factors']]]


In [ ]:
combined_df.shape

In [ ]:
combined_df['method'].value_counts()

In [ ]:
combined_df.to_csv("models_hypotheses/combined_hypotheses.csv", index=False)